# Obtención de datos

En este código se obtienen los datos de interés desde una base de datos de electrocardiogramas

In [ ]:
# paquetes
import neurokit2 as nk # incluye herramientas de análisis y detección de ondas de ECG
import pandas as pd # para trabajar con DataFrames
import numpy as np # para manejar tipos de números
import math # para operaciones especiales
import statistics # para operaciones especiales
import os # para acciones del sistema operativo

In [ ]:
# creamos una función que nos regrese la información principal de cada ECG
def analiza(id_ecg, categoria, senal, sr=100):
    # usamos un try-except para las señales conflictivas
    try:
        # primero analizamos la información
        senal_limpia = nk.ecg_clean(senal, sampling_rate=sr, method='neurokit') # quitamos el ruido de la señal
        aux_rc, picos_R = nk.ecg_peaks(senal_limpia, sampling_rate=sr) # ubicamos los picos de las ondas R
        _, picos_otros = nk.ecg_delineate(senal_limpia, picos_R, sampling_rate=sr, method="peak") # ubicamos los otros picos
    
        # segundo organizamos los valores
        ritmo_cardiaco = nk.ecg_rate(aux_rc, sampling_rate=sr)
        picos_R = picos_R['ECG_R_Peaks'] # pasamos el dato de cuando ocurre el pico a lista
        picos_R = [senal_limpia[x] for x in picos_R if not math.isnan(x)] # vemos la altura del pico / quitamos NaN's
        picos_R = [x.astype(float) for x in picos_R] # transformamos a float para evitar problemas con el formato
        picos_T = picos_otros['ECG_T_Peaks']
        picos_T = [senal_limpia[x] for x in picos_T if not math.isnan(x)]
        picos_T = [x.astype(float) for x in picos_T]
        picos_P = picos_otros['ECG_P_Peaks']
        picos_P = [senal_limpia[x] for x in picos_P if not math.isnan(x)]
        picos_P = [x.astype(float) for x in picos_P]
    
        # tercero sacamos los resultados
        max_RC = max(ritmo_cardiaco)                # los valores RC están en latidos por minuto (lpm).
        min_RC = min(ritmo_cardiaco)
        media_RC = statistics.mean(ritmo_cardiaco)
        sd_RC = statistics.stdev(ritmo_cardiaco)
        max_P = max(picos_P)                        # los valores de aqui para abajo están en mV.
        media_P = statistics.mean(picos_P)
        min_P = min(picos_P)
        sd_P = statistics.stdev(picos_P)
        max_R = max(picos_R)
        media_R = statistics.mean(picos_R)
        min_R = min(picos_R)
        sd_R = statistics.stdev(picos_R)
        max_T = max(picos_T)
        media_T = statistics.mean(picos_T)
        min_T = min(picos_T)
        sd_T = statistics.stdev(picos_T)

        # cuarto ordenamos los resultados
        res = [id_ecg, categoria, 
               min_RC, media_RC, max_RC, sd_RC, 
               min_P, media_P, max_P, sd_P,
               min_R, media_R, max_R, sd_R, 
               min_T, media_T, max_T, sd_T]
    except:
        # en caso de no poder realizar el proceso, regresa una lista de NaN's
        res = [None for x in range(16)]
        res = [id_ecg, categoria] + res
        
    return res

In [ ]:
# creamos una función que explore los archivos .csv de la carpeta con ECG
# y nos regrese un resumen de cada ECG como un renglón de un Data Frame
def explorar(direccion, tipo = ".csv"):

    # creamos el dataframe ya con los títulos
    df = pd.DataFrame(columns = ["id_ecg", "categoria",
                                 "min_RC", "media_RC", "max_RC", "sd_RC",
                                 "min_P", "media_P", "max_P", "sd_P", 
                                 "min_R", "media_R", "max_R", "sd_R", 
                                 "min_T", "media_T", "max_T", "sd_T"])
    i = 0 # para saber en que renglón vamos
    for root, dirs, files in os.walk(direccion): # os.walk va a ingresar a cada carpeta de la dirección y regresar el nombre de los archivos dentro
        for name in files: # para cada archivo...
            if name.endswith(tipo): # si el archivo es .csv...
                id_ecg = name[8:-4] # el nombre es 'patient_####.csv' entonces hacemos slicing para quitar "patient_" y ".csv".
                cat = root[13:] # el root es 'Datos_Leonel\carpeta' y queremos solo la carpeta, hacemos slicing.
                archivo = root + "\\" + name # la dirección del archivo se obtiene concatenando root y file_name
                senales = pd.read_csv(archivo, index_col='Unnamed: 0') # las señales están guardadas en formato .csv 
                senal_II = senales.II # Queremos la señal DII
                renglon = analiza(id_ecg=id_ecg, categoria = cat, senal=senal_II, sr=100) # aplicamos 'analiza' a DII
                df.loc[i,:] = renglon # concatena el resultado de 'analiza' al final del df
                i += 1 # contabilizamos el renglón
    return df # retornamos el dataframe

In [ ]:
# ejecutamos y guardamos el df para acceder a él más fácilmente
# solo como seguridad, (para evitar repetir el proceso en vano)
# agregamos una llave que debe ajustarse a True si se quiere volver a calcular el dataframe

recalcular_df = False # llave

if recalcular_df:
    df1 = explorar(direccion='Datos_Leonel')
    df2 = pd.read_csv('ptbxl_database.csv')
    df1 = df1.astype({'id_ecg':float})
    df1 = df1.merge(df2[['ecg_id','patient_id','age','sex','height','weight']],
                      how='left', left_on='id_ecg',right_on='ecg_id',sort=True)
    df1.drop(labels='ecg_id', axis = 1, inplace=True)
    df1.to_csv('df_datos.csv')
    print("datos analizados y guardados :)")

In [ ]:
# ahora si, accedemos al df de datos para ver qué tiene:
pd.set_option('display.max_columns', 100)
df = pd.read_csv('df_datos.csv', index_col='Unnamed: 0')
df.head(10)